In [ ]:
!pip install yfinance > log.txt
!pip install mplfinance
!pip install matplotlib
!pip install backtesting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 2.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173804 sha256=f9fa405a1c567e016817b9718125f3110d5de1d233769c3c32fb8453289a10d8
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [ ]:
import yfinance as yf
from google.colab import files
import pandas as pd
from datetime import datetime
import matplotlib
import mplfinance as mpf
from scipy.sparse import data

yf.pdr_override()
num = input()

start_date = datetime(int(input("開始日期\n西元年:")), int(input("月:")), int(input("日:")))
end_date = datetime(int(input("結束日期\n西元年")), int(input("月:")), int(input("日:")))
stock = yf.download(f"{num}.TW",start=start_date,end=end_date)
'''
stock.to_csv('stock.csv', encoding = 'utf-8-sig')
files.download('output.csv')#下載檔案
'''
print(stock)

2330
開始日期
西元年:2022
月:01
日:01
結束日期
西元年2022
月:12
日:31
[*********************100%***********************]  1 of 1 completed
             Open   High    Low  Close   Adj Close    Volume
Date                                                        
2022-01-03  619.0  632.0  618.0  631.0  614.044250  69089158
2022-01-04  645.0  656.0  644.0  656.0  638.372437  79434666
2022-01-05  669.0  669.0  646.0  650.0  632.533691  69593809
2022-01-06  638.0  646.0  636.0  644.0  626.694946  53210211
2022-01-07  643.0  646.0  632.0  634.0  616.963562  38249908
...           ...    ...    ...    ...         ...       ...
2022-12-26  454.0  457.0  453.5  456.5  454.043457   8793306
2022-12-27  458.0  462.5  457.0  457.0  454.540771  10264001
2022-12-28  451.0  452.0  446.0  451.0  448.573059  32578949
2022-12-29  444.0  449.5  442.5  446.0  443.599976  23020759
2022-12-30  455.0  456.5  448.5  448.5  446.086517  21683478

[246 rows x 6 columns]


In [ ]:
Kfrom backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
class SmaCross(Strategy):
    n1 = 5
    n2 = 20
    def init(self):
        self.fast_line = self.I(SMA, self.data.Close, self.n1)
        self.slow_line = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.fast_line, self.slow_line):
            """
            print(
                f"{self.data.index[-1]} Buy: Price: {self.data.Close[-1]}, Slow: {self.slow_line[-5:]}, Fast: {self.fast_line[-5:]}"
            )
            """
            self.buy()
        elif crossover(self.slow_line, self.fast_line):
            """
            print(
                f"{self.data.index[-1]} Sell: Price: {self.data.Close[-1]}, Slow: {self.slow_line[-5:]}, Fast: {self.fast_line[-5:]}"
            )
            """
            self.position.close()
print("回測結果：\n")
test = Backtest(
    stock,
    SmaCross,
    cash=10000,
    commission=0.004,
    exclusive_orders=True,
    trade_on_close=True,
)
result = test.run()
test.plot()
print(result,"\n")
opz = test.optimize(
    n1 = range(5,10,20),
    n2 = range(10,20,60),
    maximize="Return [%]",
    constraint = lambda p: p.n1 < p.n2
    )
opz
print(opz._strategy)
test.plot

回測結果：



Start                     2022-01-03 00:00:00
End                       2022-12-30 00:00:00
Duration                    361 days 00:00:00
Exposure Time [%]                   30.487805
Equity Final [$]                    10141.242
Equity Peak [$]                     10708.242
Return [%]                            1.41242
Buy & Hold Return [%]              -28.922345
Return (Ann.) [%]                    1.447117
Volatility (Ann.) [%]               16.688049
Sharpe Ratio                         0.086716
Sortino Ratio                        0.157572
Calmar Ratio                         0.107267
Max. Drawdown [%]                   -13.49082
Avg. Drawdown [%]                   -4.308178
Max. Drawdown Duration      230 days 00:00:00
Avg. Drawdown Duration       46 days 00:00:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                      12.619068
Worst Trade [%]                     -6.589365
Avg. Trade [%]                    

Backtest.optimize:   0%|          | 0/1 [00:00<?, ?it/s]

Start                     2022-01-03 00:00:00
End                       2022-12-30 00:00:00
Duration                    361 days 00:00:00
Exposure Time [%]                   41.463415
Equity Final [$]                    10460.552
Equity Peak [$]                     10586.552
Return [%]                            4.60552
Buy & Hold Return [%]              -28.922345
Return (Ann.) [%]                    4.720461
Volatility (Ann.) [%]               22.139043
Sharpe Ratio                         0.213219
Sortino Ratio                        0.386417
Calmar Ratio                         0.185507
Max. Drawdown [%]                  -25.446282
Avg. Drawdown [%]                   -6.306454
Max. Drawdown Duration      272 days 00:00:00
Avg. Drawdown Duration       63 days 00:00:00
# Trades                                    8
Win Rate [%]                             25.0
Best Trade [%]                      24.186797
Worst Trade [%]                    -16.316875
Avg. Trade [%]                    

<bound method Backtest.plot of <backtesting.backtesting.Backtest object at 0x7fa947b83ee0>>